In [1]:
# PREPROCESSING CODE FOR IMAGING DATA with Behaviour
# Creates CSV, aligned the time events, runs suite2p and calculate dff for all cells
# Creates an analysis folder & saves necessary files for each recording session
# # This code creates necessary behaviour & imaging files
# It should work for any given task recorded in Dual2p or Packer 1 scope
# Requires MATLAB integration, and uses getBehavData matlab function
# 
# 30/08/2025 HA
%reload_ext autoreload
%autoreload 2

#make data file
print( "Takes <30s. However, it might take >1 mins or so if network is busy...")
import glob
import sys,os, glob, shutil
import pandas as pd
from datetime import datetime
import numpy as np
import main_funcs as mfun
import plot_funcs as pfun
import utils_funcs as utils # utils is from Vape - catcher file: 
import matplotlib.pyplot as plt
from scipy.stats import zscore
import pickle
import time
from LakLabAnalysis.Utility.extract_paq_events import extract_paq_data_frame
import LakLabAnalysis.Utility.utils_funcs as utils_laklab
from tifffile import TiffFile, imread, imwrite
import imageio.v2 as imageio
print('All libraries are loaded correctly.')
# set matlab API
import matlab.engine
eng = matlab.engine.start_matlab()
print('Matlab engine is set correctly.')


Takes <30s. However, it might take >1 mins or so if network is busy...
All libraries are loaded correctly.
Matlab engine is set correctly.


CHANGE the animal ID  & selection criteria. Performance works or 2AFC contrast taks.

In [29]:
# Get the list to extract the files for further analysis
animalList= ['MAT057','MAT058', 'MAT066', 'MAT067']
#'MAT038','MAT039', 'MAT045', 'MAT046',
#,'HA011','HAT009','HAT010,'MBL015','MBL014',]
info = mfun.analysis(animalList=animalList)
info.recordingList = info.recordingList[
    (info.recordingList['performance'] > 60) &
    (info.recordingList['twoP']) &
    (info.recordingList['duration'] > 20)
    ].reset_index(drop=True)
info = mfun.update_info(info)
print('Total Session fits the selection: ' +  str(info.recordingList.shape[0]))

Env: decMaking
Computer: Huriye Windows
Total Session fits the selection: 28


In [30]:
info.recordingListAll = info.recordingList

info.recordingList = info.recordingList.loc[info.recordingList['suite2Pcreated']==0, :].reset_index(drop=True)

info.recordingList 

,animalID,recordingDate,recordingID,sessionName,twoP,path,imagingTiffFileNames,sessionNameWithPath,blockName,experimentDefinition,...,stimType,analysispathname,filepathname,PAQextracted,PAQdataFound,CSVcreated,CSVpath,suite2Pcreated,dffcreated,imagingDataExtracted
0,MAT057,2025-11-10,1,2025-11-10_1_MAT057,True,Z:\MAT057\2025-11-10,Z:\MAT057\2025-11-10\TwoP\2025-11-10_t-001,Z:\MAT057\2025-11-10\1\2025-11-10_1_MAT057_Blo...,2025-11-10_1_MAT057,Grating2AFC_variableStimSize_variabledelay,...,Grating2AFC_variableStimSize_variabledelay,C:\Users\Huriye\Documents\code\sideBiasLateral...,Z:\MAT057\2025-11-10\1,0,0,0,C:\Users\Huriye\Documents\code\sideBiasLateral...,0,0,0
1,MAT058,2025-11-10,1,2025-11-10_1_MAT058,True,Z:\MAT058\2025-11-10,Z:\MAT058\2025-11-10\TwoP\2025-11-10_t-001,Z:\MAT058\2025-11-10\1\2025-11-10_1_MAT058_Blo...,2025-11-10_1_MAT058,Grating2AFC_variableStimSize_variabledelay,...,Grating2AFC_variableStimSize_variabledelay,C:\Users\Huriye\Documents\code\sideBiasLateral...,Z:\MAT058\2025-11-10\1,0,0,0,C:\Users\Huriye\Documents\code\sideBiasLateral...,0,0,0
2,MAT066,2025-11-10,1,2025-11-10_1_MAT066,True,Z:\MAT066\2025-11-10,Z:\MAT066\2025-11-10\TwoP\2025-11-10_t-001,Z:\MAT066\2025-11-10\1\2025-11-10_1_MAT066_Blo...,2025-11-10_1_MAT066,Grating2AFC_variableStimSize_variabledelay,...,Grating2AFC_variableStimSize_variabledelay,C:\Users\Huriye\Documents\code\sideBiasLateral...,Z:\MAT066\2025-11-10\1,0,0,0,C:\Users\Huriye\Documents\code\sideBiasLateral...,0,0,0
3,MAT067,2025-11-10,1,2025-11-10_1_MAT067,True,Z:\MAT067\2025-11-10,Z:\MAT067\2025-11-10\TwoP\2025-11-10_t-001,Z:\MAT067\2025-11-10\1\2025-11-10_1_MAT067_Blo...,2025-11-10_1_MAT067,Grating2AFC_variableStimSize_variabledelay,...,Grating2AFC_variableStimSize_variabledelay,C:\Users\Huriye\Documents\code\sideBiasLateral...,Z:\MAT067\2025-11-10\1,0,0,0,C:\Users\Huriye\Documents\code\sideBiasLateral...,0,0,0


In [31]:
# Extract information from raw data files - Creates frames.txt, CSV, suite2p & imaging-data.pkl 
csv_reCalculate = False # To calculate the missing CSV
suite2p_reCalculate = True # To calculate the missing suite2p folder
dff_reCalculate = True


logfile = os.path.join(info.rootPath, 'Preprocessing_log_' +  animalList[0] + '.txt')
tee = mfun.Tee(logfile)
try:
    for ind, recordingDate in enumerate(info.recordingList.recordingDate):
        if ind >-9:
            start_time = time.time()
            print(f"\n#### {ind} #### Preprocessing started for : {info.recordingList.sessionName[ind]} ",  end="\n\n")

            ###### STEP 1: PAQ ///// Extract time events from PAQ file   Z:\MBL014\2025-06-13\TwoP\2025-06-13_MBL014_3_paq_imaging_frames.txt
            print(f"Step 1: Processing paq and creating imaging_frames.txt")
            filenamePAQextracted = glob.glob(os.path.join(info.recordingList.path[ind], "**", "*paq_imaging_frames.txt"))
            
            if len(filenamePAQextracted)>0:
                print('    .txt file has found. Skipping...')
                info.recordingList.loc[ind,'PAQextracted']=1
            else:
                paq_path  = os.path.join(info.recordingList.path[ind], "**", "*.paq")
                paq_files = glob.glob(paq_path, recursive=True)
                twoPpath = info.recordingList.path[ind] + '\\TwoP'
                if len(paq_files) == 1:
                    try:
                        extract_paq_data_frame(paq_files[0], save_folder=twoPpath)
                        info.recordingList.loc[ind,'PAQextracted']=1
                    except KeyError as e:
                        info.recordingList.loc[ind,'PAQextracted']=0
                        print(f"Error processing {paq_files[0]}: {e}")
                elif len(paq_files) > 1:
                    print(f"    Multiple PAQ files found for : {info.recordingList.sessionName[ind]}")
                    print(filenamePAQextracted)
                    info.recordingList.loc[ind,'PAQextracted']=0
                    continue 
                else:
                    print(f"    No PAQ files found for : {info.recordingList.sessionName[ind]}")
                    info.recordingList.loc[ind,'PAQextracted']=0
                    continue 
            
            ##### check in analysis folder for paq-data.pkl
            filenamePAQdata = glob.glob(os.path.join(info.recordingList.analysispathname[ind], "paq-data.pkl"))
            if len(filenamePAQdata) > 0:
                print('    paq-data.pkl file has found. Skipping...')
                info.recordingList.loc[ind,'PAQdataFound']=1
            else:
                try: 
                    if not os.path.exists(info.recordingList.analysispathname.iloc[ind]) :
                        os.makedirs(info.recordingList.analysispathname.iloc[ind])

                    print('    calculating paq-data.pkl file ...')
                    paq_path  = os.path.join(info.recordingList.path[ind], "**", "*.paq")
                    paq_files = glob.glob(paq_path, recursive=True)
                    paq_data = utils.paq_read( file_path=paq_files[0], plot=True, save_path=info.recordingList.analysispathname[ind])
                    filenamePAQ_analysis = os.path.join(info.recordingList.analysispathname[ind], 'paq-data.pkl')
                    with open(filenamePAQ_analysis, 'wb') as f:
                        pickle.dump(paq_data, f)
                    del paq_data
                    info.recordingList.loc[ind,'PAQextracted']=1
                except Exception as e:
                    print(f"Error processing {paq_files[0]}: {e}")
                    info.recordingList.loc[ind,'PAQextracted']=0

            ### >>> We have PAQ file properly & now can do analysis on this recording - Lets create a folder for analysis preprocessing files
            if info.recordingList.loc[ind,'PAQextracted'] !=1:
                info.recordingList.loc[ind,'CSVcreated']=0
                continue 
        
            ###### STEP 2: CSV ///// create CSV for behaviour outcome
            print('Step 2: Creating CSV file')
            filenameCSV = info.recordingList.analysispathname[ind] + info.recordingList.sessionName[ind] + '_CorrectedeventTimes.csv'
            e_filenameCSV = [f for f in glob.glob(filenameCSV)]
            if len(e_filenameCSV)==1:
                print('    CorrectedeventTimes.csv file has found. Skipping...')
                info.recordingList.loc[ind,'CSVcreated']=1
                info.recordingList.loc[ind,'CSVpath'] = filenameCSV
            else:
                if  (not csv_reCalculate) & (info.recordingList.loc[ind,'PAQextracted'] == 0):
                    info.recordingList.loc[ind,'CSVcreated']=0
                    info.recordingList.loc[ind,'CSVpath'] = filenameCSV
                else:
                    try: 
                        info.recordingList.loc[ind,'CSVpath'] = filenameCSV
                        filenameTimeline = [f for f in glob.glob(info.recordingList.filepathname[ind]+ '\\' + info.recordingList.sessionName[ind] + '_Timeline.mat')]
                        if  (len(filenameTimeline)>0):
                            sessionProfile ='Grating2AFC'
                        else:
                            sessionProfile ='Grating2AFC_noTimeline'
                            
                        # Get behaviour trial data from Block.mat file
                        data = eng.getBehavData(info.recordingList.sessionName[ind],sessionProfile)

                        # Apply correction based on the weights to match the eventTimes
                        if (len(filenameTimeline)>0) :
                            print('    Aligning time events:', end="", flush=True)
                            # Get weights to convert from probe to behavioural timebase
                            twoPpath = info.recordingList.path[ind] + '\\TwoP'
                            sessionName = info.recordingList.sessionName[ind]
                            figsavepath = info.recordingList.analysispathname[ind]
                            dataCorrected, variance = eng.applySubtractionCorrection (data, twoPpath ,sessionName, True, figsavepath, nargout=2)
                            eng.close('all', nargout=0) 
                            info.recordingList.loc[ind,'variance'] = variance

                            # Apply correction to the signal
                            data = dataCorrected
                                
                        # Save the file
                        eng.writetable(data, filenameCSV, nargout=0)
                        print("Completed.")
                        info.recordingList.loc[ind,'CSVcreated']=1
                    except Exception as e:
                        print(str(ind) + ' !!!!! FAILED: Creating CSV: ' + info.recordingList.sessionName[ind])
                        info.recordingList.loc[ind,'CSVcreated']=0

            ### >>> Only if we have CSV file properly - then we are ready for imaging data analysis
            if info.recordingList.loc[ind,'CSVcreated'] !=1:
                info.recordingList.loc[ind, 'suite2Pcreated'] = 0
                info.recordingList.loc[ind, 'dffcreated'] = 0
                continue

            ###### STEP 3: check recording quality - extract cells ROIS with suite2p 
            print('Step 3: check FOV')
            channelID = '3' # Channel ID for GCaMP in Dual2p
            twoP_dir = os.path.dirname(info.recordingList.imagingTiffFileNames[ind])
            tiff_path    = info.recordingList.imagingTiffFileNames[ind]
            filenameTiff = glob.glob(os.path.join(tiff_path, '*Ch' + channelID + '.tif'))
            if len(filenameTiff) == 0:
                print(f"TIFF missing/invalid for index {ind}: {tiff_path}")
                info.recordingList.loc[ind, 'suite2Pcreated'] = 0
                continue

            ref_dir = os.path.join(tiff_path, "References")
            ref_matches = glob.glob(os.path.join(ref_dir, "*-Ch" + channelID + "-16bit-Reference.tif"))

            #---------   Average FOV   ----------
            if not ref_matches:
                print(f"    No reference average found in {ref_dir}")
            else:
                ref_file = ref_matches[0]  # take the first match
                # destination in analysis folder
                analysis_dir = info.recordingList.analysispathname.iloc[ind]
                avg_path = os.path.join(
                    analysis_dir,
                    os.path.basename(ref_file))
                avg_path = avg_path.replace(".tif", ".png") 

                if not os.path.exists(avg_path):
                    mfun.save_png_with_contrast(ref_file, avg_path, also_save_16bit=False)
                    #shutil.copy2(ref_file, avg_path)  # copy with metadata
                    print(f"    Copied reference avg to {avg_path}")
                else:
                    print(f"    Reference avg already exists. Skipping...")

            # --------- SUITE2P CHECK / RUN ----------
            print('Step 4: check suite2p')
            suite2p_dir = os.path.join(tiff_path, "suite2p")
            if  (not suite2p_reCalculate) and os.path.isdir(suite2p_dir):
                info.recordingList.loc[ind, 'suite2Pcreated'] = 1
                print(f"    suite2p found. Skipping..")
            elif suite2p_reCalculate:
                print(f"    suite2p not found. Running extraction for: {tiff_path}")
                try:
                    filenameENV = glob.glob(os.path.join(tiff_path, "*.env"))
                    imagingDetails = mfun.parse_pv_env(filenameENV[0])
                    mfun.suite2p_extraction(tiff_path,
                                            ops_yaml_path=info.ops_yaml_path,
                                            imagingDetails=imagingDetails)
                                        #    channel_st='Ch'+channelID)
                # mfun.suite2p_extraction(tiff_path)
                    info.recordingList.loc[ind, 'suite2Pcreated'] = 1
                except Exception as e:
                    print(f"    suite2p_extraction failed for {tiff_path}: {e}")
                # set flag based on result on disk
                info.recordingList.loc[ind, 'suite2Pcreated'] = 1
            else:
                info.recordingList.loc[ind, 'suite2Pcreated'] = 0
                print(f"    suite2p not found and not calculated.")

            # ###### STEP 4: create imaging files
            print('Step 4: calculate & create imaging.pkl')

            # Read suite2p
            s2p_path = os.path.join(info.recordingList.path[ind], 'TwoP', f"{info.recordingList.recordingDate[ind]}_t-001", 'suite2p', 'plane0')
            filenameDFF = os.path.join(info.recordingList.analysispathname[ind], 'imaging-data.pkl')
            if (not dff_reCalculate) and os.path.exists(filenameDFF):
                print(f"    imaging-data.pkl already exists. Skipping...")
                info.recordingList.loc[ind, 'dffcreated'] = 1

            elif dff_reCalculate & (os.path.exists(s2p_path)) & (info.recordingList.loc[ind, 'suite2Pcreated'] == 1):
                print('    Loading suite2p data...')
                ops = np.load(os.path.join(s2p_path, 'ops.npy'), allow_pickle=True)
                ops = ops.item()
                FrameNums = ops['frames_per_file']
                filelist = ops['filelist']
                isCell = np.load(os.path.join(s2p_path, 'iscell.npy'), allow_pickle=True)

                # Load the suite2p
                flu_raw_subtracted, spks, stat = utils.s2p_loader(s2p_path)
                flu = utils.dfof2(flu_raw_subtracted)

                # Cut each session & save it in the analysis-session folder
                imaging_data = {
                    "n_frames": FrameNums,
                    "flu": flu,
                    "spks": spks,
                    "stat": stat,
                }
                
                print(f"{flu.shape[0]} cells; {flu.shape[1]} frames")
                with open(filenameDFF, 'wb') as f:
                    pickle.dump(imaging_data, f)
                print(f"    Successfully saved imaging data.")
                create_FOV_withSelectedCells(isCell, ops, s2p_path)
                print(f"    Successfully saved FOV with selected cells.")
                calculateHeatmapsForRewardedStimulus()
                print(f"    Successfully saved heatmaps.")
                info.recordingList.loc[ind, 'dffcreated'] = 1
            else:
                print(f"Suite2p path not found: {s2p_path}")
                info.recordingList.loc[ind, 'dffcreated'] = 0   
            end_time = time.time()  
            elapsed = time.time() - start_time
            print(f"{ind} Successfully completed in {elapsed:.1f} seconds\n") 
        # except Exception as e:
        #     print(str(ind) + ' !!!!! FAILED: ' + info.recordingList.sessionName[ind])
        #     print(f"Error details: {e}")
        #     info.recordingList.loc[ind, 'dffcreated'] = 0   
        #     continue  

finally:
    tee.close()   # important to restore stdout/stderr and close file

# display the output
print( "Behaviour trial data extraction completed: " + 
      str(info.recordingList['dffcreated'].sum()) +"/" + str(info.recordingList.shape[0]))


#### 0 #### Preprocessing started for : 2025-11-10_1_MAT057 

Step 1: Processing paq and creating imaging_frames.txt
Saved file: Z:\MAT057\2025-11-10\TwoP\2025-11-10_MAT057_imaging_frames.txt
Saved file: Z:\MAT057\2025-11-10\TwoP\2025-11-10_MAT057_reward_frames.txt
    calculating paq-data.pkl file ...
Step 2: Creating CSV file
    Aligning time events:Completed.
Step 3: check FOV
    Copied reference avg to C:\Users\Huriye\Documents\code\sideBiasLateralisation\analysis\2025-11-10_MAT057_1\2025-11-10_t-001-Ch3-16bit-Reference.png
Step 4: check suite2p
    suite2p not found. Running extraction for: Z:\MAT057\2025-11-10\TwoP\2025-11-10_t-001
Z:\MAT057\2025-11-10\TwoP\2025-11-10_t-001\2025-11-10_t-001_Cycle00001_Ch3.tif
29.873733883942098
Z:\MAT057\2025-11-10\TwoP\2025-11-10_t-001
CuPy version: 13.6.0
CUDA device: b'NVIDIA GeForce RTX 3080'
Ops useGPU: True
{'data_path': 'Z:\\MAT057\\2025-11-10\\TwoP\\2025-11-10_t-001', 'tiff_list': ['Z:\\MAT057\\2025-11-10\\TwoP\\2025-11-10_t-001\\2025-

In [ ]:
#Lets see how many sessions we have for each animal
info.recordingListAll = info.recordingList
info.recordingListAll['animalID'] = info.recordingListAll['animalID'].astype('category')
session_counts = info.recordingListAll['animalID'].value_counts()
print(session_counts)

In [ ]:
#10> Save info into the analysis folder
filenameINFO = info.analysisPath + '\\infoForAnalysis.pkl'
with open(filenameINFO, 'wb') as f:
    pickle.dump(info, f)
print('All should be done!!')

# Save table as CSV
recordingList = info.recordingList
recordingList.to_csv( info.analysisPath +'\\recordingList.csv', index=False)